# Task 1 of Assignment 2

**Objective:** Train separate models for pathology groups to leverage shared or unique characteristics

**Split Ratio:** 5:3:2 (50% training, 30% validation, 20% testing), applied independently per group

**Pathology Grouping:**

- Lung Diseases: Atelectasis, Consolidation, Edema, Lung Opacity, Pneumonia, Lung Lesion (6)

- Pleural-Related: Effusion, Pleural Other, Pneumothorax (3)

- Cardiac and Mediastinal: Cardiomegaly, Enlarged Cardiomediastinum (2)

- Fracture: Fracture (1)

- Support Devices: Support Devices (1)

**Key Features:**

- Data split with fixed seed (seed=42) for consistency
- NaN values converted to 0, unrecognized pathologies defaulted to 0
- Outputs dictionary with 5 groups, each containing train/validation/test datasets

In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!unzip "/content/drive/MyDrive/sph6004-2025/data/generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0.zip"

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796330/s51132520/a9e8e82e-d90e58a7-f28de941-f1e980a8-2d71737a.tfrecord  
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796330/s53980046/f96a7c3b-743943f1-f33e2e08-7325391b-3db36721.tfrecord  
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796474/s52981359/d298b8d4-f51825fb-bb351c1e-46f32718-8be91e1f.tfrecord  
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796562/s59195152/665a0427-215b7179-410d67a5-dd6b2252-90c7075a.tfrecord  
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796678/s54648930/0f3b6201-fb94a849-0af7fdcb-eeba98c8-2e6aa992.tfrecord  
  inflating: generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files/p19/p19796796/s57422647/37c6435c-cad30869-447b882a-c3fc05

### 1. Load Data

In [4]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve
from tqdm import tqdm
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set TensorFlow logging level to suppress unnecessary output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# Helper function: Load embeddings from TFRecord files
def load_embedding(embedding_path):
    raw_dataset = tf.data.TFRecordDataset([embedding_path])
    for raw_record in raw_dataset.take(1):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        embedding_feature = example.features.feature['embedding']
        embedding_values = embedding_feature.float_list.value
    embedding = torch.tensor(embedding_values)
    if torch.isnan(embedding).any() or torch.isinf(embedding).any():
        raise ValueError(f"NaN or Inf found in embedding: {embedding_path}")
    return embedding

# Custom dataset class for MIMIC chest X-ray embeddings with grouped pathologies
class MIMIC_Embed_Dataset(Dataset):
    pathology_groups = {
        "Lung Diseases": ["Atelectasis", "Consolidation", "Edema", "Lung Opacity", "Pneumonia", "Lung Lesion"],
        "Pleural-Related": ["Pleural Effusion", "Pleural Other", "Pneumothorax"],
        "Cardiac and Mediastinal": ["Cardiomegaly", "Enlarged Cardiomediastinum"],
        "Fracture": ["Fracture"],
        "Support Devices": ["Support Devices"]
    }

    def __init__(self, embedpath, csvpath, metacsvpath, group_name, views=["PA"], mode="train", seed=42, split_ratio=[0.5, 0.3, 0.2], unique_patients=True):
        super().__init__()
        np.random.seed(seed)
        self.group_name = group_name
        self.pathologies = self.pathology_groups[group_name]
        self.mode = mode
        self.embedpath = embedpath
        self.csv = pd.read_csv(csvpath)
        self.metacsv = pd.read_csv(metacsvpath)
        self.split_ratio = split_ratio

        self.csv = self.csv.set_index(["subject_id", "study_id"])
        self.metacsv = self.metacsv.set_index(["subject_id", "study_id"])
        self.csv = self.csv.join(self.metacsv).reset_index()

        self.csv["view"] = self.csv["ViewPosition"]
        self.limit_to_selected_views(views)

        if unique_patients:
            self.csv = self.csv.groupby("subject_id").first().reset_index()

        n_row = self.csv.shape[0]
        if self.mode == "train":
            self.csv = self.csv[:int(n_row * self.split_ratio[0])]
        elif self.mode == "valid":
            self.csv = self.csv[int(n_row * self.split_ratio[0]):int(n_row * (self.split_ratio[0] + self.split_ratio[1]))]
        elif self.mode == "test":
            self.csv = self.csv[int(n_row * (self.split_ratio[0] + self.split_ratio[1])):]
        else:
            raise ValueError(f"mode must be one of [train, valid, test], got {self.mode}")

        healthy = self.csv["No Finding"] == 1
        labels = []
        for pathology in self.pathologies:
            if pathology in self.csv.columns:
                self.csv.loc[healthy, pathology] = 0
                mask = self.csv[pathology]
            else:
                mask = pd.Series(0, index=self.csv.index)
            labels.append(mask.values)
        self.labels = np.asarray(labels).T.astype(np.float32)
        self.labels[np.isnan(self.labels)] = 0
        self.labels = np.where((self.labels != 0) & (self.labels != 1), 0, self.labels)


        self.csv = self.csv.reset_index(drop=True)
        self.csv["offset_day_int"] = self.csv["StudyDate"]
        self.csv["patientid"] = self.csv["subject_id"].astype(str)

    def limit_to_selected_views(self, views):
        if not isinstance(views, list):
            views = [views]
        if '*' in views:
            views = ["*"]
        self.views = views
        self.csv["view"] = self.csv["view"].fillna("UNKNOWN")
        if "*" not in views:
            self.csv = self.csv[self.csv["view"].isin(self.views)]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {}
        sample["idx"] = idx
        sample["lab"] = self.labels[idx]
        subjectid = str(self.csv.iloc[idx]["subject_id"])
        studyid = str(self.csv.iloc[idx]["study_id"])
        dicom_id = str(self.csv.iloc[idx]["dicom_id"])
        embed_file = os.path.join(self.embedpath, f"p{subjectid[:2]}", f"p{subjectid}", f"s{studyid}", f"{dicom_id}.tfrecord")
        sample["embedding"] = load_embedding(embed_file)
        return sample

# Function to initialize datasets
def initialize_mimic_datasets(embedpath, csvpath, metacsvpath, seed=42, split_ratio=[0.5, 0.3, 0.2]):
    dataset_dict = {}
    group_names = ["Lung Diseases", "Pleural-Related", "Cardiac and Mediastinal", "Fracture", "Support Devices"]

    for group in group_names:
        dataset_dict[group] = {
            "train": MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, group, mode="train", seed=seed, split_ratio=split_ratio),
            "valid": MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, group, mode="valid", seed=seed, split_ratio=split_ratio),
            "test": MIMIC_Embed_Dataset(embedpath, csvpath, metacsvpath, group, mode="test", seed=seed, split_ratio=split_ratio)
        }

    return dataset_dict

# Function to print dataset details
def print_dataset_details(datasets, num_samples=3):
    """
    Print details about the datasets to understand their structure and content.

    Args:
        datasets (dict): Dictionary of datasets for each pathology group
        num_samples (int): Number of samples to print per dataset
    """
    for group_name, splits in datasets.items():
        print(f"\n=== {group_name} ===")
        for split_name, dataset in splits.items():
            print(f"\n{split_name} Dataset:")
            print(f"Size: {len(dataset)} samples")
            print(f"Pathologies: {dataset.pathologies}")
            print(f"Label shape: {dataset.labels.shape}")

            # Print label distribution
            label_means = np.mean(dataset.labels, axis=0)
            print("Label distribution (mean per pathology):")
            for pathology, mean in zip(dataset.pathologies, label_means):
                print(f"  {pathology}: {mean:.3f}")

            # Print a few samples
            print(f"\nFirst {min(num_samples, len(dataset))} samples:")
            for i in range(min(num_samples, len(dataset))):
                sample = dataset[i]
                print(f"Sample {i}:")
                print(f"  Index: {sample['idx']}")
                print(f"  Labels: {sample['lab']}")
                print(f"  Embedding shape: {sample['embedding'].shape}")
                print(f"  Embedding (first 5 values): {sample['embedding'][:5]}")
            print("-" * 50)

# Example usage
embedpath = "/content/generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files"
csvpath = "/content/drive/MyDrive/sph6004-2025/data/mimic-cxr-2.0.0-chexpert.csv"
metacsvpath = "/content/drive/MyDrive/sph6004-2025/data/mimic-cxr-2.0.0-metadata.csv"

datasets = initialize_mimic_datasets(
    embedpath=embedpath,
    csvpath=csvpath,
    metacsvpath=metacsvpath,
    seed=42,
    split_ratio=[0.5, 0.3, 0.2]
)

# Print details about the datasets
print_dataset_details(datasets, num_samples=3)




=== Lung Diseases ===

train Dataset:
Size: 22814 samples
Pathologies: ['Atelectasis', 'Consolidation', 'Edema', 'Lung Opacity', 'Pneumonia', 'Lung Lesion']
Label shape: (22814, 6)
Label distribution (mean per pathology):
  Atelectasis: 0.088
  Consolidation: 0.018
  Edema: 0.025
  Lung Opacity: 0.118
  Pneumonia: 0.046
  Lung Lesion: 0.029

First 3 samples:
Sample 0:
  Index: 0
  Labels: [0. 0. 0. 0. 0. 0.]
  Embedding shape: torch.Size([1376])
  Embedding (first 5 values): tensor([ 0.1257, -1.8030,  1.2843, -1.8088,  0.1278])
Sample 1:
  Index: 1
  Labels: [0. 0. 0. 0. 0. 0.]
  Embedding shape: torch.Size([1376])
  Embedding (first 5 values): tensor([-0.0282, -1.4541,  1.1770, -2.3273,  0.1422])
Sample 2:
  Index: 2
  Labels: [0. 0. 0. 0. 0. 0.]
  Embedding shape: torch.Size([1376])
  Embedding (first 5 values): tensor([-0.7325, -2.4499,  1.2821, -2.6194,  0.5554])
--------------------------------------------------

valid Dataset:
Size: 13688 samples
Pathologies: ['Atelectasis', 'Co

Below is a use case for calling data, the content on the right side of the equal sign can be directly called.

In [8]:
# Load Lung Diseases
lung_diseases_train = datasets["Lung Diseases"]["train"]
lung_diseases_valid = datasets["Lung Diseases"]["valid"]
lung_diseases_test = datasets["Lung Diseases"]["test"]
# Load Pleural-Related
pleural_related_train = datasets["Pleural-Related"]["train"]
pleural_related_valid = datasets["Pleural-Related"]["valid"]
pleural_related_test = datasets["Pleural-Related"]["test"]
# Load Cardiac and Mediastinal
cardiac_mediastinal_train = datasets["Cardiac and Mediastinal"]["train"]
cardiac_mediastinal_valid = datasets["Cardiac and Mediastinal"]["valid"]
cardiac_mediastinal_test = datasets["Cardiac and Mediastinal"]["test"]
#Load Fracture
fracture_train = datasets["Fracture"]["train"]
fracture_valid = datasets["Fracture"]["valid"]
fracture_test = datasets["Fracture"]["test"]
#Load Support Devices
support_devices_train = datasets["Support Devices"]["train"]
support_devices_valid = datasets["Support Devices"]["valid"]
support_devices_test = datasets["Support Devices"]["test"]

In [5]:
import numpy as np
import pandas as pd
import os
import joblib
import torch
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import roc_auc_score, classification_report, precision_recall_curve

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

embedpath = "/content/generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files"
csvpath = "/content/drive/MyDrive/sph6004-2025/data/mimic-cxr-2.0.0-chexpert.csv"
metacsvpath = "/content/drive/MyDrive/sph6004-2025/data/mimic-cxr-2.0.0-metadata.csv"

datasets = initialize_mimic_datasets(
    embedpath=embedpath,
    csvpath=csvpath,
    metacsvpath=metacsvpath,
    seed=42,
    split_ratio=[0.5, 0.3, 0.2]
)

for group_name in datasets.keys():
    train_dataset = datasets[group_name]['train']
    assert len(train_dataset.pathologies) == train_dataset.labels.shape[1], \
        f"Pathologies mismatch in {group_name}: {len(train_dataset.pathologies)} vs {train_dataset.labels.shape[1]}"

model_config = {
    'n_estimators': 200,
    'max_depth': 15,
    'class_weight': 'balanced',
    'random_state': 42,
    'n_jobs': -1
}

def train_group_model(group_name, datasets):
    print(f"\n{'='*40}")
    print(f"Training Model for {group_name}")
    print(f"{'='*40}")

    train_dataset = datasets[group_name]['train']
    val_dataset = datasets[group_name]['valid']
    test_dataset = datasets[group_name]['test']

    def extract_features(dataset):
        embeddings = []
        for idx in range(len(dataset)):
            embedding = dataset[idx]['embedding'].numpy()
            embeddings.append(embedding)
        return np.array(embeddings), dataset.labels

    X_train, y_train = extract_features(train_dataset)
    X_val, y_val = extract_features(val_dataset)
    X_test, y_test = extract_features(test_dataset)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    base_model = RandomForestClassifier(**model_config)
    model = MultiOutputClassifier(base_model, n_jobs=-1)

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(
        model, X_train, y_train, cv=cv,
        scoring=lambda est, X, y: roc_auc_score(
            y,
            np.column_stack([p[:, -1] if p.shape[1] > 1 else p[:, 0] for p in est.predict_proba(X)]),
            average='macro'
        ),
        n_jobs=-1
    )
    print(f"Cross-val AUC: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

    model.fit(X_train, y_train)
    joblib.dump(model, f'{group_name.replace(" ", "_")}_model.pkl')

    def get_proba(proba_list):
        return np.column_stack([
            p[:, -1] if p.shape[1] > 1 else p[:, 0]
            for p in proba_list
        ])

    val_probs = get_proba(model.predict_proba(X_val))
    val_auc = roc_auc_score(y_val, val_probs, average='macro')
    print(f"Validation AUC: {val_auc:.4f}")

    optimal_thresholds = []
    for i in range(y_val.shape[1]):
        precision, recall, thresholds = precision_recall_curve(y_val[:, i], val_probs[:, i])
        f1 = 2 * precision * recall / (precision + recall + 1e-8)
        if len(f1) > 0:
            best_threshold = thresholds[np.argmax(f1)]
        else:
            best_threshold = 0.5
        optimal_thresholds.append(best_threshold)

    test_probs = get_proba(model.predict_proba(X_test))
    y_pred = (test_probs > np.array(optimal_thresholds)).astype(int)

    print("\nTest Classification Report:")
    if y_test.shape[1] == len(train_dataset.pathologies):
        adjusted_target_names = train_dataset.pathologies
    else:
        adjusted_target_names = [f'Class_{i}' for i in range(y_test.shape[1])]
        print(f"⚠️ Warning: Using auto-generated target names for {group_name}")
        print(f"Expected {y_test.shape[1]} classes, got {len(train_dataset.pathologies)} pathologies")

    report_args = {
        'y_true': y_test,
        'y_pred': y_pred,
        'target_names': adjusted_target_names,
        'digits': 4,
        'zero_division': 0
    }
    if group_name in ["Fracture", "Support Devices"]:
        report_args['labels'] = [1]
        if len(adjusted_target_names) > 1:
            adjusted_target_names = [adjusted_target_names[1]]
            report_args['target_names'] = adjusted_target_names

    print(classification_report(**report_args))

    return model

trained_models = {}
for group_name in datasets.keys():
    model = train_group_model(group_name, datasets)
    trained_models[group_name] = model

print("\nTraining completed for all groups!")


Training Model for Lung Diseases
Cross-val AUC: 0.8396 ± 0.0081
Validation AUC: 0.8395

Test Classification Report:
               precision    recall  f1-score   support

  Atelectasis     0.3590    0.5567    0.4365       855
Consolidation     0.1315    0.2892    0.1808       166
        Edema     0.3158    0.4454    0.3696       229
 Lung Opacity     0.3578    0.5393    0.4302      1120
    Pneumonia     0.1774    0.4849    0.2598       431
  Lung Lesion     0.1942    0.2472    0.2175       271

    micro avg     0.2882    0.4902    0.3630      3072
    macro avg     0.2560    0.4271    0.3157      3072
 weighted avg     0.3030    0.4902    0.3713      3072
  samples avg     0.0695    0.1031    0.0761      3072


Training Model for Pleural-Related
Cross-val AUC: 0.8974 ± 0.0125
Validation AUC: 0.8976

Test Classification Report:
                  precision    recall  f1-score   support

Pleural Effusion     0.5599    0.6550    0.6037       742
   Pleural Other     0.0691    0.2840  